In [1]:
# Import Package
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

Import your csv or excel file (Adjust the code accordingly). Rename the columns for easier access. Take a look at your dataframe and understand the meaning of each column.

In [8]:
# Import Data
measure = pd.read_csv("./Cookie weight.csv", usecols=np.arange(0,4))
measure.rename(columns={"Reading (unit = g)" : "Reading", "Cookie" : "Part"}, inplace=True)
measure

,Run Order,Part,Operator,Reading
0,1,2,2,53
1,2,4,3,53
2,3,1,1,54
3,4,1,2,54
4,5,1,3,52
5,6,3,3,52
6,7,3,3,52
7,8,1,1,52
8,9,3,2,52
9,10,2,2,52


In [9]:
# Calculate Rbar and xbb
oplist = [1, 2, 3]
partlist = [1, 2, 3, 4]
for op in oplist:
    Op = measure[measure.Operator == op]
    R = []
    xbar = []
    for part in partlist:
        data = Op.loc[Op.Part == part, "Reading"]
        R.append(max(data) - min(data))
        xbar.append(np.mean(data))
    print("Operator %s Rbar: %.4f / xbb: %.4f" % (op, np.mean(R), np.mean(xbar)))


Operator 1 Rbar: 0.7500 / xbb: 52.2500
Operator 2 Rbar: 0.7500 / xbb: 52.5000
Operator 3 Rbar: 0.5000 / xbb: 52.1667


In [4]:
Rbb = (0 + 0.25 + 0)/3 # Copy the result from above output
Rxbb = 51.5 - 51.25 # Find the max Rbar and min Rbar
d2 = 1.693 # n=3

# Calculate sigma
sigma_repeat = Rbb/d2
sigma_reproduce = Rxbb/d2
sigma_gauge = np.sqrt(sigma_repeat**2 + sigma_reproduce**2)
sigma_total = np.std(measure["Reading"], ddof=1)
sigma_part = np.sqrt(sigma_total**2 - sigma_gauge**2)

ResultTable = pd.DataFrame(columns=["Sigma", "Variance", "Ratio"])
ResultTable.loc[len(ResultTable)] = [sigma_gauge, sigma_gauge**2, (sigma_gauge**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_repeat, sigma_repeat**2, (sigma_repeat**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_reproduce, sigma_reproduce**2, (sigma_reproduce**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_part, sigma_part**2, (sigma_part**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_total, sigma_total**2, (sigma_total**2/sigma_total**2)*100]
ResultTable.index = ["sigma_gauge", "sigma_repeat", "sigma_reproduce", "sigma_part", "sigma_total"]
print(ResultTable)

                    Sigma  Variance       Ratio
sigma_gauge      0.155655  0.024228    5.311742
sigma_repeat     0.049222  0.002423    0.531174
sigma_reproduce  0.147667  0.021806    4.780568
sigma_part       0.657191  0.431900   94.688258
sigma_total      0.675372  0.456128  100.000000


In [5]:
# ANOVA - Full Model
model = ols('Reading ~ C(Part) + C(Operator) + C(Part):C(Operator)', data=measure).fit()
AnovaTable = sm.stats.anova_lm(model, type=2)
print(AnovaTable)

                       df     sum_sq   mean_sq             F        PR(>F)
C(Part)               3.0  15.924831  5.308277  25144.469298  4.318177e-42
C(Operator)           2.0   0.033450  0.016725     79.223684  2.684625e-11
C(Part):C(Operator)   6.0   0.001128  0.000188      0.890351  5.174056e-01
Residual             24.0   0.005067  0.000211           NaN           NaN


In [6]:
# Define p, o, n
p = 4 # 4 cookies
o = 3 # 3 operators
n = 3 # 3 measurements

# Copy MS from ANOVA table
MSp = 4.777778
MSo = 0.194444
MSpo = 0.194444
MSe =  0.027778


# Calculate sigma
sigma_part = np.sqrt((MSp - MSpo)/(o*n))
sigma_o = np.sqrt((MSo - MSpo)/(p*n))
sigma_po = np.sqrt((MSpo - MSe)/(n))
sigma_repeat = np.sqrt(MSe)
sigma_reproduce = np.sqrt(sigma_po**2 + sigma_o**2)
sigma_gauge = np.sqrt(sigma_repeat**2 + sigma_reproduce**2)
sigma_total = np.sqrt(sigma_gauge**2 + sigma_part**2)

ResultTable = pd.DataFrame(columns=["Sigma", "Variance", "Ratio"])
ResultTable.loc[len(ResultTable)] = [sigma_gauge, sigma_gauge**2, (sigma_gauge**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_repeat, sigma_repeat**2, (sigma_repeat**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_reproduce, sigma_reproduce**2, (sigma_reproduce**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_part, sigma_part**2, (sigma_part**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_total, sigma_total**2, (sigma_total**2/sigma_total**2)*100]
ResultTable.index = ["sigma_gauge", "sigma_repeat", "sigma_reproduce", "sigma_part", "sigma_total"]
ResultTable

,Sigma,Variance,Ratio
sigma_gauge,0.288675,0.083333,14.062498
sigma_repeat,0.166667,0.027778,4.687537
sigma_reproduce,0.235702,0.055555,9.374961
sigma_part,0.713624,0.509259,85.937502
sigma_total,0.769800,0.592593,100.000000


Use reduce model if the interaction term is not significant. (p-value > $\alpha$)

In [7]:
# Anova - Reduce Model
model = ols('Reading ~ C(Part) + C(Operator)', data=measure).fit()
AnovaTable = sm.stats.anova_lm(model, type=2)
print(AnovaTable)

               df     sum_sq   mean_sq             F        PR(>F)
C(Part)       3.0  15.924831  5.308277  25708.246637  3.143570e-51
C(Operator)   2.0   0.033450  0.016725     81.000000  8.077936e-13
Residual     30.0   0.006194  0.000206           NaN           NaN


In [8]:
# Define p, o, n
p = 4 # 4 cookies
o = 3 # 3 operators
n = 3 # 3 measurements

# Copy MS from ANOVA table
MSp = 4.777778
MSo = 0.194444
MSe =  0.061111


# Calculate sigma
sigma_part = np.sqrt((MSp - MSe)/(o*n))
sigma_reproduce = np.sqrt((MSo - MSe)/(p*n))
sigma_repeat = np.sqrt(MSe)
sigma_gauge = np.sqrt(sigma_repeat**2 + sigma_reproduce**2)
sigma_total = np.sqrt(sigma_gauge**2 + sigma_part**2)

ResultTable = pd.DataFrame(columns=["Sigma", "Variance", "Ratio"])
ResultTable.loc[len(ResultTable)] = [sigma_gauge, sigma_gauge**2, (sigma_gauge**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_repeat, sigma_repeat**2, (sigma_repeat**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_reproduce, sigma_reproduce**2, (sigma_reproduce**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_part, sigma_part**2, (sigma_part**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_total, sigma_total**2, (sigma_total**2/sigma_total**2)*100]
ResultTable.index = ["sigma_gauge", "sigma_repeat", "sigma_reproduce", "sigma_part", "sigma_total"]



ResultTable

,Sigma,Variance,Ratio
sigma_gauge,0.268742,0.072222,12.11178
sigma_repeat,0.247206,0.061111,10.24843
sigma_reproduce,0.105409,0.011111,1.86335
sigma_part,0.723930,0.524074,87.88822
sigma_total,0.772202,0.596296,100.00000
